<a href="https://colab.research.google.com/github/ogut77/CTV/blob/main/SVRTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
bts_sup = timeseries_to_supervised(bts,1)
bts_sup = bts_sup.iloc[1:,:]   # delete the line because x0 don't have antecedant
train, test = split_data(bts_sup)

# sacling data
scaler_in = MinMaxScaler()  #  for inputs
scaler_out = MinMaxScaler()  # for outputs

X_train = scaler_in.fit_transform(train[:,0].reshape(-1,1))
y_train = scaler_out.fit_transform(train[:,1].reshape(-1,1))

X_test = scaler_in.transform(test[:,0].reshape(-1,1))
y_test = scaler_out.transform(test[:,1].reshape(-1,1))


param_grid = {"C": np.linspace(10**(-2),10**3,100),
             'gamma': np.linspace(0.0001,1,20)}

mod = SVR(epsilon = 0.1,kernel='rbf')
model = GridSearchCV(estimator = mod, param_grid = param_grid,
                                   scoring = "neg_mean_squared_error",verbose = 0)

best_model = model.fit(X_train, y_train.ravel())

#prediction
predicted_tr = model.predict(X_train)
predicted_te = model.predict(X_test)

# inverse_transform because prediction is done on scaled inputs
predicted_tr = scaler_out.inverse_transform(predicted_tr.reshape(-1,1))
predicted_te = scaler_out.inverse_transform(predicted_te.reshape(-1,1))

#plot
forcast = np.concatenate((predicted_tr,predicted_te))
real = np.concatenate((train[:,1],test[:,1]))
plt.plot(real, color = 'blue', label = 'Real Erlangs')
plt.plot(forcast,"--", linewidth=2,color = 'red', label = 'Predicted Erlangs')
plt.title('Erlangs Prediction--'+data_set.columns[choice])
plt.xlabel('Time')
plt.ylabel('Erlangs')
plt.legend()
plt.show()


#error
print("MSE: ", mse(real,forcast), " R2: ", r2_score(real,forcast))
print(best_model.best_params_)